# Prolific study management

Author: Ingolf Becker, i.becker@ucl.ac.uk.

This notebook is available at [github.com/watercrossing/prolific-scripting](https://github.com/watercrossing/prolific-scripting).

In this notebook we manage the entire lifecycle of a survey study on [Prolific](https://prolific.com).
We create 96 separate prolific studies with specific filters to create a complex stratified sample, representative of the UK population by age groups, gender and area of residence. The progress of the studies is monitored from this notebook, and once completed, participant responses are checked for validity and payment to participants are released as required.

## Notes
* For anonymity purposes, all identifying identifiers (in particular, all 24 character Prolific IDs) have been randomly replaced.
* Several cells (those that cost money) have been modified to include `if False:` guards, to ensure that executing this notebook again will not cost money.
* While I tried to not edit cells retrospectively, this notebook has been edited for presentation and clarity before publication. In particular, various API calls that are specific to my account (and my projects) will not work. It is intended as an educational tool to showcase the use of Prolific's API for reproducible stratified sampling.
* The API keys need to be set in the `.env` file before importing the `APIs` library.

## Stratified sampling calculations

This worksheet calculates the required number of participants in each stratified group based on [The office of national statistics Population estimates](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates).
We are stratifying by Male/Female, age group and region.

True representative sampling requires extensive statistics based on [acceptable error margins and confidence levels](http://www.raosoft.com/samplesize.html) and depends on the subsequent analysis method. Please refer to textbooks on sampling for more detail.

While we cannot claim that our sample is representative, stratified sampling does allow us to avoid sampling biases, which are a serious problem for survey-based research.

In [ ]:
import copy
import math
import re
import time
from io import StringIO

import httpx
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from sklearn import linear_model

import APIs

In [ ]:
def loadWithRetry(fp):
    while True:
        try:
            return load_workbook(fp)
        except PermissionError:
            print("\rPlease close Excel for file %s" % fp, end="")
            time.sleep(1)

In [ ]:
popEstimates = loadWithRetry("ukpopestimatesmid2021on2021geographyfinal.xlsx")

In [ ]:
relevantSheets = [("Females", "MYE2 - Females"), ("Males", "MYE2 - Males")]
relevantRegions = [
    "North East",
    "North West",
    "Yorkshire and the Humber",
    "East Midlands",
    "West Midlands",
    "East",
    "London",
    "South East",
    "South West",
    "Wales",
    "Scotland",
    "Northern Ireland",
]
ageGroups = [(18, 24), (25, 44), (45, 64), (65, 999)]

In [ ]:
index = pd.MultiIndex.from_product([["Females", "Males"], relevantRegions], names=["gender", "region"])

In [ ]:
data = pd.DataFrame(index=index, columns=["%d-%d" % x for x in ageGroups])

In [ ]:
columns = [x.value for x in popEstimates[relevantSheets[0][1]][next(iter(popEstimates[relevantSheets[0][1]].tables.values())).ref][0]]
columns = [x if x != "90+" else "90" for x in columns]

In [ ]:
for gender, tableName in relevantSheets:
    table = popEstimates[tableName][next(iter(popEstimates[tableName].tables.values())).ref]
    for region in relevantRegions:
        for row in table:
            if row[1].value.lower() == region.lower():
                for lower, upper in ageGroups:
                    count = 0
                    for colname, cell in zip(columns, row):
                        try:
                            if int(colname) >= lower and int(colname) <= upper:
                                count += int(cell.value)
                        except ValueError:
                            pass
                    data.loc[(gender, region), "%d-%d" % (lower, upper)] = count

In [ ]:
data

18-24    25-44    45-64  65-999
gender  region                                                    
Females North East                110463   331243   362273  293819
        North West                311640   979734   974124  753035
        Yorkshire and the Humber  237135   711662   718427  565367
        East Midlands             208563   622132   651577  513276
        West Midlands             247612   776280   765155  606757
        East                      229999   838398   840577  677639
        London                    396280  1554723  1069743  582137
        South East                352844  1213091  1248077  989746
        South West                225755   693281   774623  692456
        Wales                     124529   381203   421527  358053
        Scotland                  218876   729437   770439  589120
        Northern Ireland           74484   251712   251891  177547
Males   North East                113630   313835   345954  250638
        North West                312639   935027   952255  641876
        Yorkshire and the Humber  238589   681328   700339  482069
        East Midlands             214953   599063   635810  444453
        West Midlands             256656   740539   752826  516719
        East                      243240   797944   816436  573443
        London                    379596  1433675  1019671  468306
        South East                369632  1149362  1208287  826756
        South West                237036   672013   740864  589661
        Wales                     131035   363545   401319  307739
        Scotland                  225481   717139   719686  484741
        Northern Ireland           79307   239830   243123  151678

In [ ]:
print("Total number 18 and over:", data.sum().sum())

Total number 18 and over: 53188204


In [ ]:
## sanity check with manual calculation
assert data.sum().sum() == 53188204

### Calculate required sample size

Min number of participants to recruit at least one person from each area

In [ ]:
sample = math.ceil(data.sum().sum() / data.min().min() / 2)
sample

358

In [ ]:
representative = ((data / data.sum().sum()) * sample).astype(float).round(0).astype(int)
print("Min is", representative.min().min())
representative

Min is 1


18-24  25-44  45-64  65-999
gender  region                                               
Females North East                    1      2      2       2
        North West                    2      7      7       5
        Yorkshire and the Humber      2      5      5       4
        East Midlands                 1      4      4       3
        West Midlands                 2      5      5       4
        East                          2      6      6       5
        London                        3     10      7       4
        South East                    2      8      8       7
        South West                    2      5      5       5
        Wales                         1      3      3       2
        Scotland                      1      5      5       4
        Northern Ireland              1      2      2       1
Males   North East                    1      2      2       2
        North West                    2      6      6       4
        Yorkshire and the Humber      2      5      5       3
        East Midlands                 1      4      4       3
        West Midlands                 2      5      5       3
        East                          2      5      5       4
        London                        3     10      7       3
        South East                    2      8      8       6
        South West                    2      5      5       4
        Wales                         1      2      3       2
        Scotland                      2      5      5       3
        Northern Ireland              1      2      2       1

### Only by age and gender
Just to illustrate the possibilities

In [ ]:
data.groupby(level="gender").sum()

,18-24,25-44,45-64,65-999
gender,,,,
Females,2738180,9082896,8848433,6798952
Males,2801794,8643300,8536570,5738079


In [ ]:
sample = 100

In [ ]:
(data.groupby(level="gender").sum() / data.sum().sum() * sample).astype(float).round(0).astype(int)

,18-24,25-44,45-64,65-999
gender,,,,
Females,5,17,17,13
Males,5,16,16,11


# Set up study

The following code assume that we have already manually set up a study on Prolific ("basestudy" / `templateStudy`) which has the general filters we want to apply in place.

We then retrieve the json that describes that template study and modify it to match the desired stratified parameters by code. This is substantially easier than creating the study using JSON.

In [ ]:
prol = APIs.ProlificAPI()
assert prol.get("users/me/").status_code == 200

In [ ]:
studies = prol.get("projects/dd7b4af8f0ebf2ea523d26dd/studies/")

In [ ]:
for study in studies.json()["results"]:
    print(f"{study['id']}, {study['status']}: {study['name']} ({study['internal_name']})")

00bf3f10a0967e4ea537318e, COMPLETED: Banking Fraud Survey (University College London) (APP Fraud Survey Pilot 1)
db32ada469618c2697e4dddf, COMPLETED: Banking Fraud Survey (University College London) (APP Fraud Survey (Pilot))
52db8ac6b7306c6c0d75af63, UNPUBLISHED: APP Fraud Survey (app-fraud-basestudy)
f929b3214397f3df3a5ce82a, UNPUBLISHED: APP Fraud Survey (app-fraud-1)
891f3e4fd38070830f46adae, UNPUBLISHED: APP Fraud Survey (app-fraud-2)


In [ ]:
## cleaning up left over test studies
for study in studies.json()["results"]:
    if study["status"] == "UNPUBLISHED" and "basestudy" not in study["internal_name"] and "app-fraud-" in study["internal_name"]:
        deletion = prol.delete(f"studies/{study['id']}/")
        assert deletion.status_code == 204

In [ ]:
templateStudy = prol.get("studies/52db8ac6b7306c6c0d75af63/")

In [ ]:
templateStudy.json()

{'id': '52db8ac6b7306c6c0d75af63',
 'name': 'APP Fraud Survey',
 'description': "<p>This\n study is being undertaken by researchers from University College London\n (UCL). Before proceeding, please carefully read the following.</p><p>Thank\n you for your interest in this research. We are interested in how people\n experience online banking and fraud. You do not have to have been a \nvictim of banking fraud to participate in this study.&nbsp;</p><p>In this \nstudy, you will be asked to answer survey questions about your online \nbanking experience and will be presented with several banking scenarios.\n The base survey takes around 20 minutes to complete, but it may take \nlonger as your answers may cause additional branches to be shown. In \nthese cases we will award additional bonus payments to compensate for \nyour additional time.</p><p>The survey features several attention \nchecks, in-line with Prolific's recommendations. We will ask you to \nreturn your submission in case of faile

To reproduce our research study, take the above json (you may have to tweak some prolfific IDs to match your account) and post it to the [`studies/` endpoint](https://docs.prolific.com/docs/api-docs/public/#tag/Studies/operation/CreateStudy).

In [ ]:
areas = """North East
North West
Yorkshire and the Humber
East Midlands
West Midlands
East
London
South East
South West
Wales
Scotland
Northern Ireland""".splitlines()
url = "https://qualtrics.ucl.ac.uk/jfe/form/SV_qualtrics-study-id?PROLIFIC_PID={{%PROLIFIC_PID%}}&STUDY_ID={{%STUDY_ID%}}&SESSION_ID={{%SESSION_ID%}}&g="

In [ ]:
i = 0
if "createdStudies" in locals():
    assert len(locals()["createdStudies"]) == 0
else:
    createdStudies = {}  # (agegroup, gender, region) = (str(studyNum), studyID, eligible_participant_count or -1 if not available yet)

# representative is the table a couple of cells above with the required participant counts for stratified sampling by age, gender & region
for agegroup, data in representative.items():
    for (gender, region), limit in data.items():
        print(f"Creating study {i}: {agegroup}, {gender}, {region}", end="")
        duplicated = prol.post("studies/52db8ac6b7306c6c0d75af63/clone/") , {"block_previous_participants": False})
        assert duplicated.status_code == 200
        studyID = duplicated.json()["id"]
        print(f" {studyID}", end="")
        createdStudies[(agegroup, gender, region)] = (str(i), studyID, -1)
        newfilters = copy.deepcopy(duplicated.json()["filters"])
        
        # update newfilters to match the sample we are currently recruiting for
        [x["selected_values"] for x in newfilters if x["filter_id"] == "sex"][0].remove("0" if gender == "Females" else "1")
        lowerAge, upperAge = [int(x[:2]) for x in agegroup.split("-")]
        [x for x in newfilters if x["filter_id"] == "age"][0]["selected_range"] = {"lower": lowerAge, "upper": upperAge}
        [x for x in newfilters if x["filter_id"] == "current-uk-area-of-residence"][0]["selected_values"] = [str(areas.index(region))]
        update = {
            "name": "APP Fraud Survey",
            "internal_name": "app-fraud-" + str(i),
            "external_study_url": url + str(i),
            "naivety_distribution_rate": 1,
            "total_available_places": limit,
            "filters": newfilters,
        }
        updated = prol.patch(f"studies/{studyID}/", update)

        # check that the updated study indeed has the requested sample characteristics
        assert updated.status_code == 200
        j = updated.json()
        assert j["total_available_places"] == limit
        assert len([x["selected_values"] for x in j["filters"] if x["filter_id"] == "current-uk-area-of-residence"][0]) == 1
        newAgeRange = [x["selected_range"] for x in j["filters"] if x["filter_id"] == "age"][0]
        assert newAgeRange["lower"] == lowerAge
        assert newAgeRange["upper"] == upperAge
        print(f" with {j['eligible_participant_count']} eligible participants.")
        createdStudies[(agegroup, gender, region)] = (str(i), studyID, j["eligible_participant_count"])
        i += 1

Creating study 0: 18-24, Females, North East fcb441649497ee568655023a with 53 eligible participants.
Creating study 1: 18-24, Females, North West cc9365eef7e91eeecb27b6d8 with 155 eligible participants.
Creating study 2: 18-24, Females, Yorkshire and the Humber 6a2e5fff8f02f322ac88be21 with 126 eligible participants.
Creating study 3: 18-24, Females, East Midlands 8a46ee77f9420993fb56af27 with 119 eligible participants.
Creating study 4: 18-24, Females, West Midlands 76b4a8c783b31178379ac767 with 120 eligible participants.
Creating study 5: 18-24, Females, East f965b127d48465d45c65aaa6 with 120 eligible participants.
Creating study 6: 18-24, Females, London b2ccc286e4f09bec0ccb14dd with 241 eligible participants.
Creating study 7: 18-24, Females, South East 7df6e9d90b55027f2e194383 with 221 eligible participants.
Creating study 8: 18-24, Females, South West 3f2774d04c80ffc4e82d02e6 with 148 eligible participants.
Creating study 9: 18-24, Females, Wales 34c7d1af6b2ef163fe35d553 with 55 

In [ ]:
## To delete the newly created studies for testing purposes
if False:
    for key, (group, studyID, eligibleCount) in list(createdStudies.items()):
        deletion = prol.delete(f"studies/{studyID}/")
        assert deletion.status_code == 204
        del createdStudies[key]

## Publish studies

In [ ]:
if False:
    for key, (group, studyID, eligibleCount) in createdStudies.items():
        pub = prol.post(f"studies/{studyID}/transition/", {"action": "publish"})
        assert pub.status_code == 200

## Get status of studies in the project

In [ ]:
studies = prol.get("projects/dd7b4af8f0ebf2ea523d26dd/studies/")

In [ ]:
for study in studies.json()["results"]:
    print(f"{study['id']}, {study['status']}: {study['name']} ({study['internal_name']})")

00bf3f10a0967e4ea537318e, COMPLETED: Banking Fraud Survey (University College London) (APP Fraud Survey Pilot 1)
db32ada469618c2697e4dddf, COMPLETED: Banking Fraud Survey (University College London) (APP Fraud Survey (Pilot))
52db8ac6b7306c6c0d75af63, UNPUBLISHED: APP Fraud Survey (app-fraud-basestudy)
fcb441649497ee568655023a, AWAITING REVIEW: APP Fraud Survey (app-fraud-0)
cc9365eef7e91eeecb27b6d8, AWAITING REVIEW: APP Fraud Survey (app-fraud-1)
6a2e5fff8f02f322ac88be21, AWAITING REVIEW: APP Fraud Survey (app-fraud-2)
8a46ee77f9420993fb56af27, AWAITING REVIEW: APP Fraud Survey (app-fraud-3)
76b4a8c783b31178379ac767, AWAITING REVIEW: APP Fraud Survey (app-fraud-4)
f965b127d48465d45c65aaa6, AWAITING REVIEW: APP Fraud Survey (app-fraud-5)
b2ccc286e4f09bec0ccb14dd, AWAITING REVIEW: APP Fraud Survey (app-fraud-6)
7df6e9d90b55027f2e194383, AWAITING REVIEW: APP Fraud Survey (app-fraud-7)
3f2774d04c80ffc4e82d02e6, AWAITING REVIEW: APP Fraud Survey (app-fraud-8)
34c7d1af6b2ef163fe35d553, AWAI

## Steps to do post completion
 * Verify that people that failed the attention checks are correctly processed according to Prolific policy
 * Get response data from Qualtrics, and ensure that everyone who was tracked by Prolific also correctly appears in Qualtrics, and vice versa.
 * The survey contained a couple of yes/no questions which triggered participants to answer branches. We want to analyse the additional time it took participants on these different branches to ensure that all participants area fairly compensated for their time
 * check consistency of responses to pre-screeners
 * message participants if their responses to demographic questions didn't match their prolific pre-screeners
 * issue bonus payments as required

In [ ]:
prolExport = []
for study in studies.json()["results"]:
    if rem := re.match("app-fraud-(\d+)", study["internal_name"]):
        print(f"\rExporting prolific demographics for study {study['id']}, {study['status']}: {study['name']} ({study['internal_name']})", end="")
        responses = prol.get(f"studies/{study['id']}/export/")
        assert responses.status_code == 200
        proldata = pd.read_csv(StringIO(responses.text))
        proldata["group"] = rem.groups()[0]
        proldata["studyID"] = study["id"]
        prolExport.append(proldata)
prolExport = pd.concat(prolExport)

Exporting prolific demographics for study bebc85c1bf9e820558db83ff, AWAITING REVIEW: APP Fraud Survey (app-fraud-95)

In [ ]:
prolExport.columns

Index(['Submission id', 'Participant id', 'Status',
       'Custom study tncs accepted at', 'Started at', 'Completed at',
       'Reviewed at', 'Archived at', 'Time taken', 'Completion code',
       'Total approvals', 'Current uk area of residence',
       'Years lived in current country of residence',
       'Highest education level completed', 'Bank account',
       'Uk main bank account', 'Age', 'Sex', 'Ethnicity simplified',
       'Country of birth', 'Country of residence', 'Nationality', 'Language',
       'Student status', 'Employment status', 'group', 'studyID'],
      dtype='object')

In [ ]:
len(prolExport)

412

In [ ]:
prolExport["Completion code"].unique()

array(['CYGE989O', nan, 'C15V8CP4'], dtype=object)

C15V8CP4 is 'failed attention check'. We still want to reward them for their time.

In [ ]:
prolExport[(prolExport["Completion code"] == "C15V8CP4") & (prolExport["Status"] == "AWAITING REVIEW")][["Participant id", "group"]]

,Participant id,group
3,ecdec5e0c54505b68c23571f,46
5,dfd0e6c1af31832791c30abf,79


These participants are (incorrectly) shown as AWAITING REVIEW despite using the "failed attention tests" completion code. [Need to contact support](https://researcher-help.prolific.com/hc/en-gb/articles/360009223553-Prolific-s-Attention-and-Comprehension-Check-Policy#h_01FS4DYYVP24GDYK7D0A8PSYF8) to have these status' changed. 

I send the following message to Prolific support:

    Subject: Participant marked as "AWAITING REVIEW" despite failing attention checks and completing study with the "failed attention check" completion code
    
    Good morning,
    
    I am just processing the responses for a study I collected last week. I had set the study up to automatically redirect participants to Prolific with a special completion code for participants that failed 2 attention checks. Out of ~400 responses, 6 participants failed 2 attention checks. 4 of these are automatically shown as "RETURNED", but two are marked as "AWAITING REVIEW".
    
    The help guide states to contact you to 'help returning these submission'. The participants are
    ecdec5e0c54505b68c23571f in study app-fraud-46, and
    dfd0e6c1af31832791c30abf in study app-fraud-79.
    Could you please change the return codes of these participants to “RETURNED”?


In [ ]:
prolExport[(prolExport["Completion code"] == "C15V8CP4")]

,Submission id,Participant id,Status,Custom study tncs accepted at,Started at,Completed at,Reviewed at,Archived at,Time taken,Completion code,...,Sex,Ethnicity simplified,Country of birth,Country of residence,Nationality,Language,Student status,Employment status,group,studyID
0,c97749863cd048cec5187ff5,73225eafc992562fa72241e8,RETURNED,Not Applicable,2024-02-26T16:55:51.798000Z,2024-02-26T17:24:37.157000Z,NaN,2024-03-04T10:12:41.412004Z,NaN,C15V8CP4,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,32,9b894b6e23131a6b037f0f0c
4,8967a97cd0c9c782391b4965,31d0a1a5fa33b004a6795296,RETURNED,Not Applicable,2024-02-26T17:08:31.269000Z,2024-02-26T17:46:24.799000Z,NaN,2024-03-04T10:12:43.169479Z,NaN,C15V8CP4,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,34,5241b700d0b66eb62a22b682
3,7953f26318c1c160f0787632,ecdec5e0c54505b68c23571f,AWAITING REVIEW,Not Applicable,2024-02-26T17:05:29.927000Z,2024-02-26T17:20:10.223000Z,NaN,2024-02-26T17:20:25.099558Z,881.0,C15V8CP4,...,Male,White,United Kingdom,United Kingdom,United Kingdom,English,No,Full-Time,46,963d4b9598f2822ecee4cf87
4,3cf6dd02377ca48b6eb198fd,bd3927c79c0b5fde5f14dad5,RETURNED,Not Applicable,2024-02-26T17:20:29.715000Z,2024-02-26T17:53:59.157000Z,NaN,2024-03-04T10:13:04.911918Z,NaN,C15V8CP4,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,61,0ddcaa7e5628f739e311d1e6
3,a861bb9e87361202c6aad1f2,d9e625d5a4584b3cde0f1f48,RETURNED,Not Applicable,2024-02-26T17:11:32.693000Z,2024-02-26T17:46:02.496000Z,NaN,2024-03-04T10:13:08.468896Z,NaN,C15V8CP4,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,66,16cb2c3dd5b48a6fd48a6f0f
5,d3fbd979ce63bc4f28422587,dfd0e6c1af31832791c30abf,AWAITING REVIEW,Not Applicable,2024-02-26T16:59:48.414000Z,2024-02-26T17:19:45.214000Z,NaN,2024-02-26T17:20:02.305742Z,1197.0,C15V8CP4,...,Female,White,United Kingdom,United Kingdom,United Kingdom,English,No,"Not in paid work (e.g. homemaker', 'retired or...",79,ffeb10d1877cfa17a4e5af4a


### pay bonus payment to those that failed attention checks

In [ ]:
failedButPay = prolExport[(prolExport["Completion code"] == "C15V8CP4") & (prolExport.Status == "RETURNED")]

In [ ]:
failedButPay

,Submission id,Participant id,Status,Custom study tncs accepted at,Started at,Completed at,Reviewed at,Archived at,Time taken,Completion code,...,Sex,Ethnicity simplified,Country of birth,Country of residence,Nationality,Language,Student status,Employment status,group,studyID
0,c97749863cd048cec5187ff5,73225eafc992562fa72241e8,RETURNED,Not Applicable,2024-02-26T16:55:51.798000Z,2024-02-26T17:24:37.157000Z,NaN,2024-03-04T10:12:41.412004Z,NaN,C15V8CP4,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,32,9b894b6e23131a6b037f0f0c
4,8967a97cd0c9c782391b4965,31d0a1a5fa33b004a6795296,RETURNED,Not Applicable,2024-02-26T17:08:31.269000Z,2024-02-26T17:46:24.799000Z,NaN,2024-03-04T10:12:43.169479Z,NaN,C15V8CP4,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,34,5241b700d0b66eb62a22b682
4,3cf6dd02377ca48b6eb198fd,bd3927c79c0b5fde5f14dad5,RETURNED,Not Applicable,2024-02-26T17:20:29.715000Z,2024-02-26T17:53:59.157000Z,NaN,2024-03-04T10:13:04.911918Z,NaN,C15V8CP4,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,61,0ddcaa7e5628f739e311d1e6
3,a861bb9e87361202c6aad1f2,d9e625d5a4584b3cde0f1f48,RETURNED,Not Applicable,2024-02-26T17:11:32.693000Z,2024-02-26T17:46:02.496000Z,NaN,2024-03-04T10:13:08.468896Z,NaN,C15V8CP4,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,66,16cb2c3dd5b48a6fd48a6f0f


In [ ]:
bonuses = {}

In [ ]:
payBonuses = False
for studyID in failedButPay.studyID.unique():
    topay = failedButPay[(failedButPay.studyID == studyID)].copy()
    topay["amount"] = "2.00"
    csv_bonuses = topay[["Participant id", "amount"]].to_csv(header=False, index=False, lineterminator="\n")
    if studyID not in bonuses and payBonuses:
        req = prol.post("submissions/bonus-payments/", {"study_id": studyID, "csv_bonuses": csv_bonuses})
        assert req.status_code == 201
        bonuses[studyID] = req.json()

In [ ]:
bonuses

{'9b894b6e23131a6b037f0f0c': {'id': '16b79f086b8585a1ff6da699',
  'study': '9b894b6e23131a6b037f0f0c',
  'amount': 200.0,
  'fees': 66.0,
  'vat': 13.0,
  'total_amount': 279.0},
 '5241b700d0b66eb62a22b682': {'id': '6f1fe639cdce56ce9e46e19a',
  'study': '5241b700d0b66eb62a22b682',
  'amount': 200.0,
  'fees': 66.0,
  'vat': 13.0,
  'total_amount': 279.0},
 '0ddcaa7e5628f739e311d1e6': {'id': '997247bc9a5b748cc5106abd',
  'study': '0ddcaa7e5628f739e311d1e6',
  'amount': 200.0,
  'fees': 66.0,
  'vat': 13.0,
  'total_amount': 279.0},
 '16cb2c3dd5b48a6fd48a6f0f': {'id': 'd7be6740c3dd91b7ee317f48',
  'study': '16cb2c3dd5b48a6fd48a6f0f',
  'amount': 200.0,
  'fees': 66.0,
  'vat': 13.0,
  'total_amount': 279.0}}

In [ ]:
sum(x["total_amount"] for x in bonuses.values()) / 100

11.16

In [ ]:
paidBonuses = []

In [ ]:
for bonus in bonuses.values():
    if payBonuses and bonus["id"] not in paidBonuses:
        req = prol.post(f"bulk-bonus-payments/{bonus['id']}/pay/", {})
        assert req.status_code == 202
        paidBonuses.append(bonus["id"])

## Get data, deduplication

In [ ]:
qual = APIs.QualtricsAPI()

In [ ]:
qual.getData("SV_qualtrics-study-id", "data")

Download is being prepared... 21.51% Done.


In [ ]:
data = pd.read_csv("data/APP fraud reimbursement.csv", skiprows=range(1, 71))  ## Rows 1-71 are test and pre-study responses by manual inspection
headers = pd.read_csv("data/APP fraud reimbursement.csv", nrows=2)
assert data.loc[0, "Q109"] == "526b7326da5926f98500cf25"  ## Make sure that we have skipped correctly to the real data, this is the first real data point

In [ ]:
len(data)

397

In [ ]:
## any duplicate 'Participant id'?
(prolExport["Participant id"].value_counts() > 1).any()

False

In [ ]:
data.Q109.value_counts()[data.Q109.value_counts() > 1]

Q109
c425f7b60d30b4b2bedee290    2
Name: count, dtype: int64

In [ ]:
data[data.Q109.isin(data.Q109.value_counts()[data.Q109.value_counts() > 1].keys())]

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,timer-fv-6_Last Click,timer-fv-6_Page Submit,timer-fv-6_Click Count,PROLIFIC_PID,timings,STUDY_ID,SESSION_ID,attFailed,Create a New Field or Choose from the Drop-down...,g
360,2024-02-26 17:42:25,2024-02-26 18:56:54,IP Address,100,4468,True,2024-02-26 18:56:55,R_8Fr0r43HMyfpIid,anonymous,EN,...,0.0,3.452,0.0,c425f7b60d30b4b2bedee290,page 3 load at 1708969504006;QR~QID1~10 at 170...,21a7a7f8c3b15f5f56fae0ba,1f1d6befe8a20ca6b5be8324,1,NaN,59
388,2024-02-26 17:32:05,2024-02-26 17:41:22,IP Address,17,556,False,2024-02-27 17:41:26,R_8QirN7I24iZVBmR,anonymous,EN,...,NaN,NaN,NaN,c425f7b60d30b4b2bedee290,page 3 load at 1708969018286;QR~QID1~9 at 1708...,21a7a7f8c3b15f5f56fae0ba,1f1d6befe8a20ca6b5be8324,0,NaN,59


manually inspected the data for participants that appear more than once. Nothing suspicious here.

In [ ]:
assert data.loc[388, "Q109"] == "c425f7b60d30b4b2bedee290"
data = data.drop(index=388)

In [ ]:
assert len(data.Q109.value_counts()[data.Q109.value_counts() > 1]) == 0

In [ ]:
data[~(data["Q109"] == data["PROLIFIC_PID"])][["Q109", "PROLIFIC_PID"]]

,Q109,PROLIFIC_PID
101,3c6c4b9747ddedecc76bc0f4,3c6c4b9747ddedecc76bc0f4
270,c3cbd492a4d96f520b0e4fac@email.prolific.com,c3cbd492a4d96f520b0e4fac
325,60534,3c08de0c18e54c0f4fa83dac


That seems ok to me.

In [ ]:
assert len(data.PROLIFIC_PID.value_counts()[data.PROLIFIC_PID.value_counts() > 1]) == 0

In [ ]:
len(set(prolExport["Participant id"].values) & set(data.PROLIFIC_PID.values)), len(prolExport), len(data)

(396, 412, 396)

There seem to be some participant IDs that don't exist in both datasets. Investigate.

In [ ]:
prolExport[prolExport["Participant id"].isin(set(prolExport["Participant id"].values) - set(data.PROLIFIC_PID.values))]

,Submission id,Participant id,Status,Custom study tncs accepted at,Started at,Completed at,Reviewed at,Archived at,Time taken,Completion code,...,Sex,Ethnicity simplified,Country of birth,Country of residence,Nationality,Language,Student status,Employment status,group,studyID
1,6e0bf9977d7b28d62fc47525,fb6971f5b78d10a0401848aa,RETURNED,Not Applicable,2024-02-26T17:58:00.090000Z,NaN,NaN,2024-03-04T10:12:21.715905Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,4,76b4a8c783b31178379ac767
0,c05841f4528bfc948fb386cf,d49e11a97c3f428e608f1f7b,RETURNED,Not Applicable,2024-02-26T17:10:00.073000Z,NaN,NaN,2024-03-04T10:12:28.028655Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,13,8c8f6247197acb9326cb8a55
1,73f24cf349fd7bd84d8675a1,dbc79bbbbac9443c86368ee2,RETURNED,Not Applicable,2024-02-26T17:03:39.785000Z,NaN,NaN,2024-03-04T10:12:35.437994Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,26,bc1384dada74786005a932b2
1,a88dc4b1faf43653548de2b6,56881b396a31b126d6b72310,RETURNED,Not Applicable,2024-02-26T17:01:00.070000Z,NaN,NaN,2024-03-04T10:12:37.290699Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,28,f293fa3f3f4bdbc6e4a29a95
3,a683254d0f0f306235526691,cb2a47c9cdccd3370ee04c86,RETURNED,Not Applicable,2024-02-26T17:06:00.026000Z,NaN,NaN,2024-03-04T10:12:37.920473Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,29,aef25cebc7974df4defb41a8
7,e172669fa42ec58f15ca5b19,43b4c8498ad287162fe1a6f7,RETURNED,Not Applicable,2024-03-04T10:16:41.049000Z,NaN,NaN,NaN,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,32,9b894b6e23131a6b037f0f0c
4,64dd991813b366193bae862d,86743b092d410f93d255633b,RETURNED,Not Applicable,2024-02-26T17:15:29.942000Z,NaN,NaN,2024-03-04T10:12:54.737972Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,49,58284b1f9dde33f5cf48b0cb
0,e1a2554537e5e55fcd380524,382c70cf9f9c4948a0945a33,RETURNED,Not Applicable,2024-02-26T16:48:03.434000Z,NaN,NaN,2024-03-04T10:12:59.928431Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,56,8f51a0024b4dd35e69474c70
0,6a890edbf55fa4b3905c7602,74aa5d47c90cff6ff0efa286,RETURNED,Not Applicable,2024-02-26T17:10:33.551000Z,NaN,NaN,2024-03-04T10:13:06.221669Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,63,72336a59c27c08b8cc527f13
1,7c5cd8db039e0c05932afe0d,495f8e919de37aac2b787ecc,RETURNED,Not Applicable,2024-02-26T17:13:42.320000Z,NaN,NaN,2024-03-04T10:13:07.149522Z,NaN,NaN,...,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,CONSENT_REVOKED,64,cbc5ba987f57d5e6d9584bd9


All seem to have RETURNED or TIMED-OUT before ever starting the study. Nothing to do here.

In [ ]:
prolExport[prolExport["Participant id"].isin(set(data.PROLIFIC_PID.values) - set(prolExport["Participant id"].values))]

,Submission id,Participant id,Status,Custom study tncs accepted at,Started at,Completed at,Reviewed at,Archived at,Time taken,Completion code,...,Sex,Ethnicity simplified,Country of birth,Country of residence,Nationality,Language,Student status,Employment status,group,studyID


And nothing the other way round. So we are good to merge!

In [ ]:
merged = pd.merge(prolExport, data, left_on="Participant id", right_on="PROLIFIC_PID")
merged = merged.rename(columns={"Status_x": "ProlStatus", "Status_y": "QualStatus", "Duration (in seconds)": "Duration"})

In [ ]:
assert len([x for x in merged.columns if x.endswith("_y")]) == 0

Inspect participants without return codes and that actually finished the survey

In [ ]:
merged[merged["Completion code"].isna() & merged.Finished][["ProlStatus", "Finished", "attFailed", "Duration"]]

,ProlStatus,Finished,attFailed,Duration
166,TIMED-OUT,True,0,4538
185,TIMED-OUT,True,2,59223
210,TIMED-OUT,True,1,4286
244,TIMED-OUT,True,1,4468


The three that didn't fail too many attention checks were only ever so slightly over the max time, so they should be returned.

In [ ]:
merged[merged["Completion code"].isna() & merged.Finished & (merged.attFailed < 2)]["PROLIFIC_PID"].tolist()

['94d073d4b84725ae976f8a64',
 'ae32386991c887b34477104f',
 'c425f7b60d30b4b2bedee290']

## Inspect all completed responses

In [ ]:
merged[merged.Finished & (merged.attFailed < 2)].ProlStatus.value_counts()

ProlStatus
AWAITING REVIEW    360
TIMED-OUT            3
Name: count, dtype: int64

Ok, those three are the same from above. That's straight forward.

In [ ]:
finished = merged[merged.Finished & (merged.attFailed < 2)].copy()

In [ ]:
haveApproved = []

In [ ]:
doApprovals = False
for subID in finished[finished.ProlStatus != "APPROVED"]["Submission id"]:
    if doApprovals and subID not in haveApproved:
        req = prol.post(f"submissions/{subID}/transition/", {"action": "APPROVE"})
        assert req.status_code == 200
        haveApproved.append(subID)

## Timings by branches
Want to reward participants that had substantially longer surveys based on their answer options.

In [ ]:
branches = [
    (["Q5.2c", "Q5.2d"], "timer-scen1-3"),
    (["Q6.2c", "Q158"], "timer-scen2-2"),
    (["Q2.25", "Q2.26", "Q2.27", "Q2.28", "Q2.29"], "timer-obe-12"),
    (["Q2.34", "Q2.35", "Q2.36", "Q2.37", "Q2.38"], "timer-obe-15"),
    (["Q3.3"], "timer-afe-2"),
    (["Q3.6"], "timer-afe-4"),
    (["Q4.2"], "timer-fra-1"),
    (["Q4.4"], "timer-fra-2"),
]
all(x in finished.columns for y in branches for x in y[0])

True

In [ ]:
# non-missing item counts per branch
(~finished[[y for x in branches for y in x[0]]].isna()).sum()

Q5.2c    348
Q5.2d     34
Q6.2c    298
Q158      93
Q2.25     18
Q2.26    310
Q2.27    338
Q2.28     39
Q2.29    118
Q2.34    121
Q2.35    294
Q2.36    276
Q2.37    108
Q2.38     72
Q3.3      61
Q3.6      12
Q4.2     136
Q4.4     258
dtype: int64

In [ ]:
## Additional time spend (estimated based on linear models) on answering optional free text responses
# negative numbers don't make much sense here, and probably indicate cross-effects, but fortunately they are only two here.
# Otherwise the model output seems sensible.
additionalTime = {}
for indeps, dep in branches:
    lr = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 50))
    lr.fit(~finished[indeps].isna(), finished[dep + "_Page Submit"].fillna(0))
    print(indeps, lr.coef_, lr.intercept_, lr.alpha_)
    for ind, indT in zip(indeps, lr.coef_):
        additionalTime[ind] = max(0, indT)

['Q5.2c', 'Q5.2d'] [-1.60645818  1.16383113] 11.48086828293943 22.229964825261955
['Q6.2c', 'Q158'] [21.64942001 22.83650922] 21.708353940881516 2.329951810515372
['Q2.25', 'Q2.26', 'Q2.27', 'Q2.28', 'Q2.29'] [-11.56000372  36.07014934  43.46229239  57.76381418  34.5051093 ] 11.387769400495785 2.329951810515372
['Q2.34', 'Q2.35', 'Q2.36', 'Q2.37', 'Q2.38'] [12.07491002 27.83546421 31.11250311 48.79040996 27.07971943] 15.961882859220836 7.196856730011514
['Q3.3'] [58.87342775] 0.0836691660851443 0.42919342601287785
['Q3.6'] [36.76718908] 0.04495793687864458 0.42919342601287785
['Q4.2'] [20.48903204] 12.107125738504035 1.325711365590108
['Q4.4'] [29.3070637] 0.06816133866239227 0.244205309454865


These numbers seem very sensible - extra time spend for writing stuff and for additional questions all in the order of 30s-1min.

In [ ]:
finished["additionalTime"] = (~finished[additionalTime.keys()].isna()).replace(dict((k, {True: v}) for k, v in additionalTime.items())).sum(axis=1)

In [ ]:
basetime = (finished.Duration - finished.additionalTime).astype(float)

In [ ]:
currentPayment = 3.00
(currentPayment / (basetime / 60 / 60)).mean()

8.933865838945154

So our base-time pay rate is just under 9GBP, just as planned, great!

In [ ]:
(currentPayment / (finished.Duration / 60 / 60)).mean()

7.410937383016092

Before extra payments, we are paying everyone at £7.41 per hour. Still acceptable by Prolific's standards, but obviously they will receive more shortly.

In [ ]:
print("Cost of additional payments for extra branches: £%.2f" % (finished.additionalTime.sum() / 60 / 60 * 9))

Cost of additional payments for extra branches: £201.61


In [ ]:
bonuses = {}

In [ ]:
totals = []
payBonuses = False
for studyID in finished.studyID.unique():
    topay = finished[(finished.studyID == studyID) & (finished.additionalTime > 60)][["Participant id", "additionalTime"]].copy()
    topay["amount"] = topay.apply(lambda r: "%.2f" % (r.additionalTime * 9 / 60 / 60), axis=1)
    totals.append(topay.apply(lambda r: float("%.2f" % (r.additionalTime * 9 / 60 / 60)), axis=1))
    csv_bonuses = topay[["Participant id", "amount"]].to_csv(header=False, index=False, lineterminator="\n")
    if studyID not in bonuses and payBonuses:
        req = prol.post("submissions/bonus-payments/", {"study_id": studyID, "csv_bonuses": csv_bonuses})
        assert req.status_code == 201
        bonuses[studyID] = req.json()

In [ ]:
pd.concat(totals).sum()

201.38

In [ ]:
sum(x["total_amount"] for x in bonuses.values()) / 100

281.16

In [ ]:
paidBonuses = []

In [ ]:
for bonus in bonuses.values():
    if payBonuses and bonus["id"] not in paidBonuses:
        req = prol.post(f"bulk-bonus-payments/{bonus['id']}/pay/", {})
        assert req.status_code == 202
        paidBonuses.append(bonus["id"])

## Pre-screeners matching

We only requested responses from people that have UK bank accounts. There was a pre-existing filter on Prolific for this. We get the responses to this filter as part of the data from Prolific, but at the same time we ask participants the same question in our survey too. That way we can validate their responses, and we get an additional indicator of good data quality.

The same applies to questions on `Age`, `UK area`, `Time lived in country of residence`, and `highest education level completed?`. 

The logic below compares the answers, and messages the participants to highlight their inconsistency to them.

We could decide to throw these responses away and re-collect more data, but since none of these questions are integral to our research questions and only a small number of participants have answered incorrectly, we will keep them.

In [ ]:
finished = finished.replace({"Uk main bank account": {"Lloyds Bank": "Lloyds"}})

In [ ]:
# Sex, Age, Area of residence, years lived in country of country of residence, highest education level
# UK Main bank account, employment status (not the same scales)
toMatch = [
    (
        "What is your age?",
        "Q2.1",
        "Age",
        lambda r: int(r.iloc[1])
        in range(int(r.iloc[0].split("-")[0]) if "-" in r.iloc[0] else 70, int(r.iloc[0].split("-")[1]) + 1 if "-" in r.iloc[0] else 150),
    ),
    ("Sex", "Q2.2", "Sex", lambda r: r.iloc[1] in r.iloc[0] or "Prefer" in r.iloc[0]),
    (
        "What UK area do you currently live in?",
        "Q2.5.2",
        "Current uk area of residence",
        lambda r: r.iloc[0].strip().replace("   ", "") == r.iloc[1].strip().replace("   ", ""),
    ),
    (
        "How many years have you lived in your current country of residence?",
        "Q2.5.3",
        "Years lived in current country of residence",
        lambda r: (
            (r.iloc[0].startswith("5") and r.iloc[1].split(" ")[1] in ["5", "6", "7", "8", "9"])
            or (r.iloc[0].startswith("m") and r.iloc[1].split(" ")[1] in ["10", "15", "20", "than"])
        ),
    ),
    (
        "What is your highest education level completed?",
        "Q2.5",
        "Highest education level completed",
        lambda r: (
            ("High" in r.iloc[0] and any(s in r.iloc[1] for s in ["High", "No", "Secondary", "Technical"]))
            or ("undergraduate" in r.iloc[0] and "Undergraduate" in r.iloc[1])
            or ("postgraduate" in r.iloc[0] and "Graduate" in r.iloc[1])
            or ("Doctoral" in r.iloc[0] and "Doctorate" in r.iloc[1])
            or ("College" in r.iloc[0] and any(s in r.iloc[1] for s in ["college", "High", "Secondary"]))
            or ("undergraduate" in r.iloc[0] and "Undergraduate" in r.iloc[1])
        ),
    ),
    (
        "Which bank is your main current account with?",
        ["Q2.9_%d" % i for i in range(1, 14)] + ["Q2.9_10_TEXT"] + ["Q2.10_%d" % i for i in range(1, 13)] + ["Q2.10_11_TEXT"],
        "Uk main bank account",
        lambda r: any(r.iloc[-1] in z for z in ["Other", "Unknown"] + [x.strip() for x in r.iloc[0:-1].values if type(x) == str]),
    ),
]

In [ ]:
finished["banksCustomerAt"] = finished[toMatch[5][1]].apply(
    lambda r: ", ".join(set([x.strip() for x in r.values if type(x) == str and "Other" not in x])), axis=1
)

In [ ]:
msg = "Dear Participant,\n\
I would like to thank you for your time in taking to respond to our study on Banking Fraud last week. While reviewing your submission \
I noticed that some of your answers to demographic questions were not consistent with your responses with Prolific pre-screeners. Rest \
assured I will not be taking any further action about this and your payment will not be influenced by this, but you may want to review \
your responses to Prolific’s pre-screening questions to make sure that they are accurate. In particular:\n\n"

In [ ]:
notMatching = {}
with pd.option_context("display.max_colwidth", None):
    for pscreener, qCol, pCol, mFun in toMatch:
        print(pscreener)
        if "current account" in pscreener:
            notMatching[pscreener] = finished[["banksCustomerAt", pCol]][
                ~finished[[qCol, pCol] if type(qCol) == str else (qCol + [pCol])].apply(mFun, axis=1)
            ]
        else:
            notMatching[pscreener] = finished[[qCol, pCol]][~finished[[qCol, pCol]].apply(mFun, axis=1)]
        display(notMatching[pscreener])

What is your age?


,Q2.1,Age
197,40-44,45
240,40-44,59
286,50-54,56
352,60-64,65


Sex


,Q2.2,Sex


What UK area do you currently live in?


...

,Q2.5.3,Years lived in current country of residence
33,more than 10 years,Between 9 and 10 years
52,more than 10 years,Between 5 and 6 years
69,more than 10 years,Between 9 and 10 years
161,more than 10 years,Between 5 and 6 years
179,more than 10 years,Between 7 and 8 years
191,more than 10 years,Between 9 and 10 years
294,more than 10 years,Between 5 and 6 years
295,more than 10 years,Between 9 and 10 years


What is your highest education level completed?


,Q2.5,Highest education level completed
4,"University undergraduate (e.g., BSc)",High school diploma/A-levels
8,"University undergraduate (e.g., BSc)",Technical/community college
17,"University undergraduate (e.g., BSc)",High school diploma/A-levels
18,"University undergraduate (e.g., BSc)",High school diploma/A-levels
32,"University undergraduate (e.g., BSc)",High school diploma/A-levels
50,University postgraduate (e.g. MSc),High school diploma/A-levels
67,College/technical training,Undergraduate degree (BA/BSc/other)
81,"University undergraduate (e.g., BSc)",High school diploma/A-levels
86,University postgraduate (e.g. MSc),Undergraduate degree (BA/BSc/other)
110,College/technical training,Undergraduate degree (BA/BSc/other)


Which bank is your main current account with?


,banksCustomerAt,Uk main bank account
5,First Direct,HSBC
21,Halifax,Royal Bank of Scotland
22,"Revolut, Halifax, Monzo",Lloyds
65,HSBC,NatWest
73,"Starling, Halifax, Barclays, Monzo",NatWest
85,"Lloyds, Monzo",Barclays
106,"Revolut, Halifax",Bank of Scotland
162,"Revolut, Barclays",NatWest
179,"Wise, Halifax, Plum",Bank of Scotland
180,Nationwide,Lloyds


In [ ]:
responses = {}

In [ ]:
actuallySendMessage = False
for i in finished.index:
    m = msg
    isNonMatching = False
    for pscreener, nm in notMatching.items():
        if i in nm.index:
            isNonMatching = True
            if "current account" in pscreener:
                m += (
                    f'When we asked you to list all banks that you are a customer at, your list was "{nm.loc[i].iloc[0]}". However, to '
                    + f'Prolific\'s pre-screening question "{pscreener}" you responded with "{nm.loc[i].iloc[1]}", which is missing from the '
                    + "list you gave to us.\n\n"
                )
            else:
                m += (
                    f'For the question "{pscreener}", your answer to the survey was "{nm.loc[i].iloc[0]}", while your answer to Prolific\'s '
                    + f'pre-screening question with the same wording was "{nm.loc[i].iloc[1]}".\n\n'
                )
    m += "Kind regards"
    if isNonMatching:
        if actuallySendMessage and (finished.loc[i, "Participant id"] not in responses or responses[finished.loc[i, "Participant id"]] != 204):
            po = prol.post("messages/", {"recipient_id": finished.loc[i, "Participant id"], "body": m, "study_id": finished.loc[i, "STUDY_ID"]})
            responses[finished.loc[i, "Participant id"]] = po.status_code
            assert po.status_code == 204
            time.sleep(0.5)
        print(m)

Dear Participant,
I would like to thank you for your time in taking to respond to our study on Banking Fraud last week. While reviewing your submission I noticed that some of your answers to demographic questions were not consistent with your responses with Prolific pre-screeners. Rest assured I will not be taking any further action about this and your payment will not be influenced by this, but you may want to review your responses to Prolific’s pre-screening questions to make sure that they are accurate. In particular:

For the question "What UK area do you currently live in?", your answer to the survey was "South West, England (Gloucestershire, Wiltshire and Bristol/Bath area, Dorset and Somerset, Cornwall and Isles of Scilly, Devon)", while your answer to Prolific's pre-screening question with the same wording was "North East, England    (Tees Valley, Durham, Northumberland and Tyne and Wear)".

Kind regards
Dear Participant,
I would like to thank you for your time in taking to res

## Final checks

In [ ]:
studies = prol.get("projects/dd7b4af8f0ebf2ea523d26dd/studies/")

In [ ]:
for study in studies.json()["results"]:
    print(f"{study['id']}, {study['status']}: {study['name']} ({study['internal_name']})")

00bf3f10a0967e4ea537318e, COMPLETED: Banking Fraud Survey (University College London) (APP Fraud Survey Pilot 1)
db32ada469618c2697e4dddf, COMPLETED: Banking Fraud Survey (University College London) (APP Fraud Survey (Pilot))
52db8ac6b7306c6c0d75af63, UNPUBLISHED: APP Fraud Survey (app-fraud-basestudy)
fcb441649497ee568655023a, COMPLETED: APP Fraud Survey (app-fraud-0)
cc9365eef7e91eeecb27b6d8, COMPLETED: APP Fraud Survey (app-fraud-1)
6a2e5fff8f02f322ac88be21, COMPLETED: APP Fraud Survey (app-fraud-2)
8a46ee77f9420993fb56af27, COMPLETED: APP Fraud Survey (app-fraud-3)
76b4a8c783b31178379ac767, COMPLETED: APP Fraud Survey (app-fraud-4)
f965b127d48465d45c65aaa6, COMPLETED: APP Fraud Survey (app-fraud-5)
b2ccc286e4f09bec0ccb14dd, COMPLETED: APP Fraud Survey (app-fraud-6)
7df6e9d90b55027f2e194383, COMPLETED: APP Fraud Survey (app-fraud-7)
3f2774d04c80ffc4e82d02e6, COMPLETED: APP Fraud Survey (app-fraud-8)
34c7d1af6b2ef163fe35d553, COMPLETED: APP Fraud Survey (app-fraud-9)
b3dea64b1108d3d3

## Export data for analysis

We don't need all columns for analysis. We export a dataset that only contains the relevant columns, and drops any potentially identifying information (prolific PIDs), so that the new dataset is fully anonymous and can be released as part of our reproducible analysis pipeline.

In [ ]:
finished.columns[~finished.apply(lambda x: x.isna().all() or len(x.unique()) == 1)].to_list()

['Submission id',
 'Participant id',
 'ProlStatus',
 'Started at',
 'Completed at',
 'Archived at',
 'Time taken',
 'Completion code',
 'Total approvals',
 'Current uk area of residence',
 'Years lived in current country of residence',
 'Highest education level completed',
 'Bank account',
 'Uk main bank account',
 'Age',
 'Sex',
 'Ethnicity simplified',
 'Country of birth',
 'Nationality',
 'Language',
 'Student status',
 'Employment status',
 'group',
 'studyID',
 'StartDate',
 'EndDate',
 'Duration',
 'RecordedDate',
 'ResponseId',
 'Q_RecaptchaScore',
 'Q_RelevantIDFraudScore',
 'Q109',
 'Q1.1_Browser',
 'Q1.1_Version',
 'Q1.1_Operating System',
 'Q1.1_Resolution',
 'Q1.2_Id',
 'Q1.2_Size',
 'timer-consent_First Click',
 'timer-consent_Last Click',
 'timer-consent_Page Submit',
 'timer-consent_Click Count',
 'Q2.1',
 'Q2.2',
 'Q2.2_7_TEXT',
 'Q2.3',
 'Q2.3_9_TEXT',
 'Q2.3_10_TEXT',
 'timer-obe-0_First Click',
 'timer-obe-0_Last Click',
 'timer-obe-0_Page Submit',
 'timer-obe-0_Clic

In [ ]:
columnsToExport = [
    "Total approvals",
    "Current uk area of residence",
    "Years lived in current country of residence",
    "Highest education level completed",
    "Bank account",
    "Uk main bank account",
    "Age",
    "Sex",
    "Ethnicity simplified",
    "Country of birth",
    "Nationality",
    "Language",
    "Student status",
    "Employment status",
    "group",
    "StartDate",
    "EndDate",
    "Duration",
    "RecordedDate",
    "Q_RecaptchaScore",
    "Q_RelevantIDFraudScore",
    "Q1.1_Browser",
    "Q1.1_Version",
    "Q1.1_Operating System",
    "Q1.1_Resolution",
    "timer-consent_First Click",
    "timer-consent_Last Click",
    "timer-consent_Page Submit",
    "timer-consent_Click Count",
    "Q2.1",
    "Q2.2",
    "Q2.2_7_TEXT",
    "Q2.3",
    "Q2.3_9_TEXT",
    "Q2.3_10_TEXT",
    "timer-obe-0_First Click",
    "timer-obe-0_Last Click",
    "timer-obe-0_Page Submit",
    "timer-obe-0_Click Count",
    "Q2.4",
    "Q2.5",
    "Q2.5.2",
    "timer-obe-1_First Click",
    "timer-obe-1_Last Click",
    "timer-obe-1_Page Submit",
    "timer-obe-1_Click Count",
    "Q2.5.3",
    "Q2.6",
    "Q2.7",
    "Q2.8",
    "timer-obe-2_First Click",
    "timer-obe-2_Last Click",
    "timer-obe-2_Page Submit",
    "timer-obe-2_Click Count",
    "Q2.9_1",
    "Q2.9_2",
    "Q2.9_3",
    "Q2.9_4",
    "Q2.9_5",
    "Q2.9_6",
    "Q2.9_7",
    "Q2.9_8",
    "Q2.9_9",
    "Q2.9_12",
    "Q2.9_13",
    "Q2.9_10",
    "Q2.9_10_TEXT",
    "Q2.10_1",
    "Q2.10_2",
    "Q2.10_3",
    "Q2.10_4",
    "Q2.10_6",
    "Q2.10_7",
    "Q2.10_8",
    "Q2.10_9",
    "Q2.10_11",
    "Q2.10_11_TEXT",
    "Q2.11_5",
    "Q2.11_1",
    "Q2.11_2",
    "Q2.11_3",
    "Q2.11_4",
    "Q2.11_6",
    "Q2.11_6_TEXT",
    "Q2.12",
    "timer-obe-3_First Click",
    "timer-obe-3_Last Click",
    "timer-obe-3_Page Submit",
    "timer-obe-3_Click Count",
    "Q5.2a",
    "timer-scen1-1_First Click",
    "timer-scen1-1_Last Click",
    "timer-scen1-1_Page Submit",
    "timer-scen1-1_Click Count",
    "Q5.2b",
    "timer-scen1-2_First Click",
    "timer-scen1-2_Last Click",
    "timer-scen1-2_Page Submit",
    "timer-scen1-2_Click Count",
    "Q5.2c",
    "Q5.2d",
    "Q5.3",
    "timer-scen1-3_First Click",
    "timer-scen1-3_Last Click",
    "timer-scen1-3_Page Submit",
    "timer-scen1-3_Click Count",
    "Q5.4_1",
    "Q5.4_2",
    "Q5.4_3",
    "Q5.4_4",
    "timer-scen1-4_First Click",
    "timer-scen1-4_Last Click",
    "timer-scen1-4_Page Submit",
    "timer-scen1-4_Click Count",
    "Q5.5a",
    "timer-scen1-5_First Click",
    "timer-scen1-5_Last Click",
    "timer-scen1-5_Page Submit",
    "timer-scen1-5_Click Count",
    "Q5.5b",
    "Q5.6",
    "timer-scen1-7_First Click",
    "timer-scen1-7_Last Click",
    "timer-scen1-7_Page Submit",
    "timer-scen1-7_Click Count",
    "Q6.2a",
    "timer-scen2-1_First Click",
    "timer-scen2-1_Last Click",
    "timer-scen2-1_Page Submit",
    "timer-scen2-1_Click Count",
    "Q6.2b",
    "Q6.2c",
    "Q158",
    "timer-scen2-2_First Click",
    "timer-scen2-2_Last Click",
    "timer-scen2-2_Page Submit",
    "timer-scen2-2_Click Count",
    "Q6.3",
    "timer-scen2-3_First Click",
    "timer-scen2-3_Last Click",
    "timer-scen2-3_Page Submit",
    "timer-scen2-3_Click Count",
    "Q6.4_1",
    "Q6.4_2",
    "Q6.4_3",
    "Q6.4_4",
    "timer-scen2-4_First Click",
    "timer-scen2-4_Last Click",
    "timer-scen2-4_Page Submit",
    "timer-scen2-4_Click Count",
    "Q6.5a",
    "timer-scen2-5_First Click",
    "timer-scen2-5_Last Click",
    "timer-scen2-5_Page Submit",
    "timer-scen2-5_Click Count",
    "Q6.5b",
    "Q6.6",
    "timer-scen2-6_First Click",
    "timer-scen2-6_Last Click",
    "timer-scen2-6_Page Submit",
    "timer-scen2-6_Click Count",
    "Q2.12.1",
    "Q2.13_1",
    "Q2.13_2",
    "Q2.13_3",
    "Q2.13_4",
    "Q2.13_5",
    "Q2.14",
    "timer-obe-4_First Click",
    "timer-obe-4_Last Click",
    "timer-obe-4_Page Submit",
    "timer-obe-4_Click Count",
    "Q2.15_1",
    "Q2.15_2",
    "Q2.15_3",
    "Q2.15_4",
    "Q2.15_5",
    "Q2.15_6",
    "Q2.15_7",
    "Q2.15_8",
    "Q2.15_9",
    "Q2.15_10",
    "Q2.15_11",
    "Q2.15_12",
    "timer-obe-5_First Click",
    "timer-obe-5_Last Click",
    "timer-obe-5_Page Submit",
    "timer-obe-5_Click Count",
    "timer-obe-6_First Click",
    "timer-obe-6_Last Click",
    "timer-obe-6_Page Submit",
    "timer-obe-6_Click Count",
    "Q2.17_1",
    "Q2.17_2",
    "Q2.17_3",
    "Q2.18_1",
    "Q2.18_2",
    "Q2.18_3",
    "timer-obe-7_First Click",
    "timer-obe-7_Last Click",
    "timer-obe-7_Page Submit",
    "timer-obe-7_Click Count",
    "Q2.19",
    "Q2.20",
    "timer-obe-8_First Click",
    "timer-obe-8_Last Click",
    "timer-obe-8_Page Submit",
    "timer-obe-8_Click Count",
    "Q2.21",
    "timer-obe-9_First Click",
    "timer-obe-9_Last Click",
    "timer-obe-9_Page Submit",
    "timer-obe-9_Click Count",
    "Q2.23a",
    "Q2.23b",
    "timer-obe-10_First Click",
    "timer-obe-10_Last Click",
    "timer-obe-10_Page Submit",
    "timer-obe-10_Click Count",
    "Q2.24_1",
    "Q2.24_2",
    "Q2.24_3",
    "Q2.24_4",
    "Q2.24_5",
    "Q2.24_6",
    "timer-obe-11_First Click",
    "timer-obe-11_Last Click",
    "timer-obe-11_Page Submit",
    "timer-obe-11_Click Count",
    "Q2.25",
    "Q2.26",
    "Q2.27",
    "Q2.28",
    "Q2.29",
    "Q2.30",
    "timer-obe-12_First Click",
    "timer-obe-12_Last Click",
    "timer-obe-12_Page Submit",
    "timer-obe-12_Click Count",
    "Q2.31",
    "Q2.32a",
    "Q2.32b",
    "timer-obe-13_First Click",
    "timer-obe-13_Last Click",
    "timer-obe-13_Page Submit",
    "timer-obe-13_Click Count",
    "Q2.33_1",
    "Q2.33_2",
    "Q2.33_3",
    "Q2.33_4",
    "Q2.33_5",
    "Q2.33_6",
    "timer-obe-14_First Click",
    "timer-obe-14_Last Click",
    "timer-obe-14_Page Submit",
    "timer-obe-14_Click Count",
    "Q2.34",
    "Q2.35",
    "Q2.36",
    "Q2.37",
    "Q2.38",
    "Q2.38.1",
    "timer-obe-15_First Click",
    "timer-obe-15_Last Click",
    "timer-obe-15_Page Submit",
    "timer-obe-15_Click Count",
    "Q3.1",
    "Q3.2",
    "Q165_First Click",
    "Q165_Last Click",
    "Q165_Page Submit",
    "Q165_Click Count",
    "timer-afe-1_First Click",
    "timer-afe-1_Last Click",
    "timer-afe-1_Page Submit",
    "timer-afe-1_Click Count",
    "Q3.3",
    "timer-afe-2_First Click",
    "timer-afe-2_Last Click",
    "timer-afe-2_Page Submit",
    "timer-afe-2_Click Count",
    "Q3.5",
    "timer-afe-3_First Click",
    "timer-afe-3_Last Click",
    "timer-afe-3_Page Submit",
    "timer-afe-3_Click Count",
    "Q3.6",
    "timer-afe-4_First Click",
    "timer-afe-4_Last Click",
    "timer-afe-4_Page Submit",
    "timer-afe-4_Click Count",
    "Q4.1",
    "Q4.2",
    "Q4.3",
    "timer-fra-1_First Click",
    "timer-fra-1_Last Click",
    "timer-fra-1_Page Submit",
    "timer-fra-1_Click Count",
    "Q4.4",
    "timer-fra-2_First Click",
    "timer-fra-2_Last Click",
    "timer-fra-2_Page Submit",
    "timer-fra-2_Click Count",
    "Q7.4",
    "timer-fv-1_First Click",
    "timer-fv-1_Last Click",
    "timer-fv-1_Page Submit",
    "timer-fv-1_Click Count",
    "Q7.5_1",
    "Q7.5_2",
    "Q7.5_3",
    "Q7.5_4",
    "Q7.5_5",
    "Q7.5_6",
    "Q7.5_7",
    "Q7.5_8",
    "Q7.5_10",
    "Q7.5_11",
    "Q7.5_12",
    "timer-fv-2_First Click",
    "timer-fv-2_Last Click",
    "timer-fv-2_Page Submit",
    "timer-fv-2_Click Count",
    "Q7.6_1",
    "Q7.6_2",
    "Q7.6_3",
    "Q7.6_4",
    "Q7.6_5",
    "Q7.6_6",
    "Q7.6_7",
    "Q7.6_8",
    "Q159",
    "timer-fv-3_First Click",
    "timer-fv-3_Last Click",
    "timer-fv-3_Page Submit",
    "timer-fv-3_Click Count",
    "Q7.7",
    "Q7.7a",
    "timer-fv-4_First Click",
    "timer-fv-4_Last Click",
    "timer-fv-4_Page Submit",
    "timer-fv-4_Click Count",
    "Q7.8_1",
    "Q7.8_2",
    "Q7.8_3",
    "Q7.8_4",
    "Q7.8_5",
    "Q7.8_6",
    "Q7.8_7",
    "Q7.8_8",
    "Q7.8_9",
    "Q7.9",
    "timer-fv-5_First Click",
    "timer-fv-5_Last Click",
    "timer-fv-5_Page Submit",
    "timer-fv-5_Click Count",
    "Q7.11",
    "timer-fv-6_First Click",
    "timer-fv-6_Last Click",
    "timer-fv-6_Page Submit",
    "timer-fv-6_Click Count",
    "timings",
]

In [ ]:
finished[columnsToExport].to_csv("data/cleaned.csv", index=None)

In [ ]:
headers[[x for x in headers.columns if x in columnsToExport]].to_csv("data/cleanedQualtricsHeaders.csv", index=None)